djasjdalskdlaksdjasljd


In [1]:
with open('input.txt', 'r', encoding='utf-8') as file:
    text = file.read();

In [2]:
print(f"Dataset tamaño : {len(text)}")

Dataset tamaño : 805521


In [3]:
tokens = sorted(list(set(text)))
numero_tokens = len(tokens)
print(f"VOCABULARIO: {''.join(tokens)}")
print(numero_tokens)

VOCABULARIO: 	
 !"#$%&'()*+,-./0123456789:;<=>?@ABCDEFGHIJKLMNOPQRSTUVWXYZ[\]^_abcdefghijklmnopqrstuvwxyz{|}~ ¡ª´¿ÉÍÑÜáçéíñóúı˜̴̵̷̸̡̢̧̨̛̖̗̘̙̜̝̞̟̠̣̤̥̦̩̪̫̬̭̮̯̰̱̲̳̹̺̻̼͇͈͉͍͎͓͔͕͖͙͚̀́̂̃̄̅̆̇̈̉̊̋̌̍̎̏̐̑̒̓̔̽̾̿̀́͂̓̈́͆͊͋͌͐͑͒͗͛̕̚͘͜͝͠ͅ​‍‎‘’“”…€≥●☀☆☹♂✌❤➕️�🌈🏳🏻🏼👈👉👊👌👍🔥🗿😂😅😈😊😎😏😐😑😔😗😡😮😰😳🙃🙏🤑🤙🤡🤤🤭🤮🤷🥵🥶🥹🥺🦯🧑
267


In [4]:
char_to_int = { ch:i for i,ch in enumerate(tokens) }
int_to_char = { i:ch for i,ch in enumerate(tokens) }
encode = lambda str: [char_to_int[car] for car in str]
decode = lambda lint: [int_to_char[car] for car in lint] 

print(encode("hola"));
print(decode(encode("hola")));

[73, 80, 77, 66]
['h', 'o', 'l', 'a']


In [5]:
import torch;
data = torch.tensor(encode(text), dtype=torch.long)
print(data.shape, data.dtype)
print(data)

torch.Size([805521]) torch.int64
tensor([19, 20, 17,  ..., 66, 84,  1])


In [6]:
train_percentage = 0.7
train_total = int(train_percentage * len(data))
train_set = data[:train_total]
test_set = data[train_total:]

In [7]:
context_length = 8
train_set[:context_length+1]

tensor([19, 20, 17, 19, 20, 17, 20, 18, 14])

In [8]:
#Explicacion tienes dos arrays donde la letra n preferida de n-1 ... n - contexto tiene que predecir n + 1
contexto_actual = train_set[:context_length+1]
contexto_a_predecir = train_set[1:context_length+1]
for letras_leidas in range(context_length):
    contexto = contexto_actual[:letras_leidas+1]
    a_predecir = contexto_a_predecir[letras_leidas]
    print(f"cuando el contexto es {contexto} se busca el token {a_predecir}")
    

cuando el contexto es tensor([19]) se busca el token 20
cuando el contexto es tensor([19, 20]) se busca el token 17
cuando el contexto es tensor([19, 20, 17]) se busca el token 19
cuando el contexto es tensor([19, 20, 17, 19]) se busca el token 20
cuando el contexto es tensor([19, 20, 17, 19, 20]) se busca el token 17
cuando el contexto es tensor([19, 20, 17, 19, 20, 17]) se busca el token 20
cuando el contexto es tensor([19, 20, 17, 19, 20, 17, 20]) se busca el token 18
cuando el contexto es tensor([19, 20, 17, 19, 20, 17, 20, 18]) se busca el token 14


In [9]:
torch.manual_seed(1337)
batch_size = 4 #batches in paralel
context_length = 8 #context size

def get_batch(split):
    data = train_set if split == "train" else test_set
    context_offset_batch = torch.randint(len(data) - context_length, (batch_size, ))
    x = torch.stack([data[i:i+context_length] for i in context_offset_batch])
    y = torch.stack([data[i+1:i+context_length+1] for i in context_offset_batch])
    return x,y

tokens_in, targets = get_batch("train")
print(f"inputs {tokens_in.shape}\n{tokens_in}")
print(f"targets {targets.shape}\n{targets}")
print("=================================================")

inputs torch.Size([4, 8])
tensor([[84, 66, 83,  2, 69, 70,  2, 81],
        [17, 17, 53, 74,  2, 70, 84,  2],
        [84,  2, 78, 86, 70, 83, 85, 80],
        [ 2, 90, 66,  2, 68, 80, 79,  2]])
targets torch.Size([4, 8])
tensor([[66, 83,  2, 69, 70,  2, 81, 80],
        [17, 53, 74,  2, 70, 84,  2, 70],
        [ 2, 78, 86, 70, 83, 85, 80, 84],
        [90, 66,  2, 68, 80, 79,  2, 70]])


In [10]:
for batch in range(batch_size):
    for context_place in range(context_length):
        context = tokens_in[batch, :context_place+1]
        target_for = targets[batch, context_place]
        print(f"when inputed {context.tolist()} target is {target_for}")



when inputed [84] target is 66
when inputed [84, 66] target is 83
when inputed [84, 66, 83] target is 2
when inputed [84, 66, 83, 2] target is 69
when inputed [84, 66, 83, 2, 69] target is 70
when inputed [84, 66, 83, 2, 69, 70] target is 2
when inputed [84, 66, 83, 2, 69, 70, 2] target is 81
when inputed [84, 66, 83, 2, 69, 70, 2, 81] target is 80
when inputed [17] target is 17
when inputed [17, 17] target is 53
when inputed [17, 17, 53] target is 74
when inputed [17, 17, 53, 74] target is 2
when inputed [17, 17, 53, 74, 2] target is 70
when inputed [17, 17, 53, 74, 2, 70] target is 84
when inputed [17, 17, 53, 74, 2, 70, 84] target is 2
when inputed [17, 17, 53, 74, 2, 70, 84, 2] target is 70
when inputed [84] target is 2
when inputed [84, 2] target is 78
when inputed [84, 2, 78] target is 86
when inputed [84, 2, 78, 86] target is 70
when inputed [84, 2, 78, 86, 70] target is 83
when inputed [84, 2, 78, 86, 70, 83] target is 85
when inputed [84, 2, 78, 86, 70, 83, 85] target is 80
wh

In [11]:
import torch
import torch.nn as nn
from torch.nn import functional as F
torch.manual_seed(1337)

class BigramLanguageModel(nn.Module):
    def __init__(self, vocab_size):
        super().__init__()
        self.token_embedding_table = nn.Embedding(vocab_size, vocab_size)
    
    def forward(self, idx, targets=None):
        logits = self.token_embedding_table(idx)
        # batch by time by channel table (B, T, C)
        if targets is None:
            loss = None
        else:
            #Hay que transformarla para que funcione segun la documentacion a B*T, C
            B, T, C = logits.shape
            logits = logits.view(B*T, C)
            targets = targets.view(B*T) #o (-1)
            loss = F.cross_entropy(logits, targets)
        return logits, loss
    
    #coge B T --> lo tranorma a B T+1
    def generate(self, idx, max_new_tokens):
        #idx es el contexto B T
        for _ in range(max_new_tokens):
            #prerdicciones
            logits, loss = self(idx)
            #cogemos la ultima de todas
            #porque es lo que viene ahora
            logits = logits[:,-1,:] #B C
            #softmax a las probabilidades
            probs = F.softmax(logits, dim=-1) # B C
            #cogemos un sample de la distribucion --> SOLO 1 prediccion
            idx_next = torch.multinomial(probs, num_samples=1)# B 1
            #realizamos un append del sample a la secuencia B T+1
            idx=torch.cat((idx, idx_next), dim=1)
        return idx
    
m = BigramLanguageModel(numero_tokens)
logits, loss = m(tokens_in, targets)
print(logits.shape)
print(loss)

print(
    decode(
        m.generate(
            torch.zeros((1,1), dtype=torch.long), max_new_tokens=100
            )[0].tolist()
    )
)

torch.Size([32, 267])
tensor(5.9427, grad_fn=<NllLossBackward0>)
['\t', '͓', 'B', '̽', 'r', '●', '\\', 'j', '🙏', 's', '“', '?', ';', '̾', '\xa0', '͇', 'ú', '🤷', '@', 'í', 'G', 'T', '\\', '#', '͓', 'u', 's', '͑', '̵', '”', '\xa0', '´', '<', 'á', '🥺', '̯', '_', '̵', 'v', '🦯', '!', 'l', 'á', '🤤', 't', '͙', ' ', '😐', 'Ñ', '͖', '\t', '͌', '“', 'É', '̫', '̨', 'D', '🥺', '👈', '̼', '*', '👈', '-', '🗿', '👈', '̹', 'm', '❤', '\u200b', '➕', '̑', 'Q', '̠', '̿', '\xa0', '͔', 'M', '͗', '̩', '͠', '\\', '👍', '+', '̼', '̨', ':', '͚', '~', '9', '🗿', '$', '\U0001f979', '͓', 'ñ', 'V', '🤤', '-', '̦', '🤙', '̬', 'C']


In [12]:
optimizer = torch.optim.AdamW(m.parameters(), lr=1e-3)
#para los gradientes

In [13]:
batch_size = 32
for steps in range(10000):
    xb, yb = get_batch("train")

    #loss evaluation
    logits, loss = m(xb, yb)
    optimizer.zero_grad(set_to_none=True)
    loss.backward()
    optimizer.step()

print(loss.item())

2.2518138885498047


In [24]:
logits.size(-1)

267

In [14]:
print(
    decode(
        m.generate(
            torch.zeros((1,1), dtype=torch.long), max_new_tokens=100
            )[0].tolist()
    )
)

['\t', '.', '̗', '🏻', '<', ' ', '̞', '}', '͊', '́', 'B', 'I', 'Z', '☹', '[', '5', '/', '4', '9', '.', '0', ':', ' ', '_', 'p', 'i', 'u', 'e', 'p', 'e', 'n', 'o', '.', 'a', ' ', 'p', 'i', 'u', 'n', 'g', 'e', 'a', 'l', 't', 'o', ' ', ' ', 'G', 'i', 'u', 'e', 'l', 'a', ' ', 'v', 'a', '\n', '2', ',', ' ', 'e', ' ', 'd', '\n', ' ', '2', ':', ' ', 'r', 'e', ' ', 'p', 'u', 's', '[', 'm', 'p', 'o', '>', '\n', ' ', '-', ' ', '/', '5', '1', '?', '̒', '͇', 'Ñ', '´', '̛', '🥺', '̎', '̄', '̞', '😑', "'", '(', 'r', 'd']


In [15]:
B, T, C = 4, 8 ,32
x = torch.randn(B, T, C)
x.shape

x_bag_of_words = torch.zeros((B, T,C))
for b in range(B):
    for t in range(T):
        xprev = x[b, :t+1] #(t, C) vector 1 dimenshon
        x_bag_of_words[b,t] = torch.mean(xprev, 0)


weights = torch.tril(torch.ones(T,T))
weights = weights / torch.sum(weights, 1, keepdim=True)
xbow2 = weights @ x # B, T, T @ B, T ,C --> B T C
torch.allclose(xbow2, x_bag_of_words)

tril = torch.tril(torch.ones(T, T))
wei = torch.zeros((T, T))
wei = wei.masked_fill(tril==0, float("-inf"))
wei = F.softmax(wei, dim=-1);
xbow3 = wei @ x

In [16]:
a = torch.tril(torch.ones(3,3))
a = a / torch.sum(a, 1, keepdim=True)

In [17]:
#heads of attention

head_size = 16
key = nn.Linear(C, head_size, bias=False)
query = nn.Linear(C, head_size, bias=False)
value = nn.Linear(C, head_size, bias=False)
k = key(x) # B T 16
q = query(x) # B T 16

wei = q @ k.transpose(-2,-1) * head_size**-0.5# B T 16 @ B 16 T --> B T T 

tril = torch.tril(torch.ones(T, T))
wei = wei.masked_fill(tril==0, float("-inf"))
wei = F.softmax(wei, dim=-1)
v = value(x)
out = wei @ v

wei[:2]

tensor([[[1.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
         [0.5355, 0.4645, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
         [0.3592, 0.2387, 0.4021, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
         [0.2119, 0.3349, 0.2025, 0.2507, 0.0000, 0.0000, 0.0000, 0.0000],
         [0.2838, 0.1680, 0.1804, 0.1936, 0.1741, 0.0000, 0.0000, 0.0000],
         [0.1257, 0.1426, 0.1584, 0.2308, 0.1751, 0.1674, 0.0000, 0.0000],
         [0.1517, 0.1561, 0.1246, 0.1546, 0.1194, 0.1442, 0.1495, 0.0000],
         [0.1593, 0.1123, 0.1371, 0.0816, 0.1285, 0.1071, 0.1458, 0.1283]],

        [[1.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
         [0.3903, 0.6097, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
         [0.4576, 0.3158, 0.2266, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
         [0.3615, 0.2059, 0.2118, 0.2208, 0.0000, 0.0000, 0.0000, 0.0000],
         [0.2256, 0.2560, 0.1833, 0.1304, 0.2048, 0.0000, 0.0000, 0.0000],
         [0.1710, 0.127

In [18]:
#

In [19]:
import os

checkpoint_name = "state.pth"
state = {
    "epoch": epoch +1,
    "state_dict": m.state_dict(), #modelo,
    "optimizer": optimizer.state_dict(),
    "losslogger": loss,
}
torch.save(state, f"{checkpoint_name}")


def load_checkpoint(model, optimizer, losslogger, filename):
    starting_epoch = 0
    if os.path.isfile(filename):
        checkpoint = torch.load(filename)
        starting_epoch = checkpoint["epoch"]
        model.load_state_dict(checkpoint["state_dict"])
        optimizer.load_state_dict(optimizer["optimizer"])
        loss = checkpoint["losslogger"]
    else:
        print("no checkpoint")
    
    return model, optimizer, starting_epoch, loss


NameError: name 'epoch' is not defined